# АНАЛИЗ ЭФФЕКТИВНОСТИ НОВОГО АЛГОРИТМА ПОИСКА АНКЕТ

Вы работаете аналитиком в приложении для онлайн-знакомств. Механика приложения следующая: пользователи видят в приложении анкеты друг друга и ставят лайки или дизлайки. Если пользователи поставили друг другу лайк – это мэтч. У пользователей появляется возможность познакомиться.

Команда приложения разработала новый алгоритм для поиска наиболее подходящих анкет. Для проверки работы алгоритма провели A/B-тест. Все пользователи были разделены на две группы. Пользователи в группе с номером 0 пользовались приложением со старым алгоритмом. Пользователи в группе №1 пользовались приложением с новым алгоритмом для поиска анкет.

**Ваша задача**: оценить, действительно ли новый алгоритм улучшил качество сервиса. Для этого нужно выбрать метрики, которые отвечают за качество сервиса, и статистически сравнить эти метрики в двух группах.

В данных находится выгрузка логов взаимодействия пользователей друг с другом. Для каждой пары пользователей указаны их группа в A/B-тесте и факт мэтча.
* Group 0 - контрольная группа (пользовались приложением со старым алгоритмом);
* Group 1 - экспертиментальная группа (пользовались новым алгоритмом для поиска анкет).

**Результат вашей работы** – аналитическое заключение с ответом на вопрос, стоит ли включать новую систему поиска анкет на всех пользователей.

Импортируем необходимые библиотеки:

In [2]:
import numpy as np
import pandas as pd

import scipy.stats as ss

import seaborn as sns
import matplotlib.pyplot as plt

## Предварительный анализ данных

In [3]:
dating_data = pd.read_csv('dating_data.csv')

In [4]:
dating_data.head()

,user_id_1,user_id_2,group,is_match
0,79,91,1,1
1,716,353,1,1
2,423,677,0,0
3,658,165,1,1
4,969,155,0,1


In [8]:
# Размер файла
dating_data.shape

(14514, 4)

In [9]:
# Проверка на наличие пустых значений
dating_data.isna().sum()

user_id_1    0
user_id_2    0
group        0
is_match     0
dtype: int64

In [5]:
# Число уникальных значений 
dating_data.nunique()

user_id_1    1000
user_id_2    1000
group           2
is_match        2
dtype: int64

In [6]:
# Сколько пользователей относится к каждой из групп
dating_data.group.value_counts()

1    9722
0    4792
Name: group, dtype: int64

Видно, что число пар пользователей из экспериментальной группы больше в 2 раза по сравнению с контрольной. Возможно пользователи из экспериментальной группы имели больше желания и возможностей для взаимодействия друг с другом как раз благодаря изменению алгоритма поиска анкет.

## Посчитаем долю совпадений в группах:

In [10]:
# Группируем данные по пользователям и группам, чтобы подсчитать действия и совпадения
aggregate = (
    dating_data.groupby(['user_id_1', 'group'], as_index=False)
      .agg({
          'user_id_2': 'count',  # Подсчитываем количество действий пользователя (лайков/дизлайков)
          'is_match': 'sum'      # Суммируем количество взаимных лайков (совпадений)
      })
      .rename(columns={
          'user_id_1': 'user_id',
          'user_id_2': 'actions',
          'is_match': 'matchs_count'
      })
)

aggregate

,user_id,group,actions,matchs_count
0,1,1,24,11
1,2,1,16,7
2,3,1,16,5
3,4,0,9,2
4,5,1,22,13
...,...,...,...,...
995,996,0,8,1
996,997,0,12,1
997,998,1,18,10
998,999,0,7,2


In [13]:
# Рассчитываем долю совпадений среди действий пользователя
aggregate['matchs_share'] = round(aggregate['matchs_count'] / aggregate['actions'], 2)

# Выводим результирующий DataFrame
aggregate

,user_id,group,actions,matchs_count,matchs_share
0,1,1,24,11,0.46
1,2,1,16,7,0.44
2,3,1,16,5,0.31
3,4,0,9,2,0.22
4,5,1,22,13,0.59
...,...,...,...,...,...
995,996,0,8,1,0.12
996,997,0,12,1,0.08
997,998,1,18,10,0.56
998,999,0,7,2,0.29


In [8]:
aggregate.group.value_counts()

0    501
1    499
Name: group, dtype: int64

## Сравнение доли мэтчей между контрольной и тестовой группами


In [9]:
aggregate.query('group == 0').matchs_share.describe()

count    501.000000
mean       0.193593
std        0.134368
min        0.000000
25%        0.100000
50%        0.180000
75%        0.270000
max        0.670000
Name: matchs_share, dtype: float64

In [10]:
aggregate.query('group == 1').matchs_share.describe()

count    499.000000
mean       0.402405
std        0.114209
min        0.000000
25%        0.330000
50%        0.400000
75%        0.480000
max        0.720000
Name: matchs_share, dtype: float64

Средняя доля мэтчей в тестовой группе (= 0.4) в два раза больше, чем в контрольной (= 0.19). Однако рано делать выводы, так как пока не доказана статистическая значимость подобных различий.

Проверим, есть ли различия в средней доле мэтчей на пользователя в двух полученных группах с помощью t-критерия Уэлча, поскольку стандартные отклонения выборок не равны и обе переменные количественные:

* Н0 - различия в средней доле мэтчей статистически не значимы;
* Н1 - различия в средней доле мэтчей статистически значимы.

In [14]:
# Выполнение t-критерия Уэлча
ss.ttest_ind(aggregate.query('group == 0').matchs_share,
aggregate.query('group == 1').matchs_share,
equal_var=False)

Ttest_indResult(statistic=-26.481431782585016, pvalue=7.890669157069948e-117)

pvalue < 0.05, значит отклоняем нулевую гипотезу и делаем вывод, что средняя доля мэтчей между двумя группами статистически различается. 

## Сравнение среднего количества действий между контрольной и тестовой группами

In [15]:
# Контрольная группа
aggregate.query('group == 0').actions.describe()

count    501.000000
mean       9.564870
std        3.056515
min        1.000000
25%        8.000000
50%        9.000000
75%       12.000000
max       20.000000
Name: actions, dtype: float64

In [16]:
# Тестовая группа
aggregate.query('group == 1').actions.describe()

count    499.000000
mean      19.482966
std        2.991489
min       11.000000
25%       18.000000
50%       20.000000
75%       22.000000
max       29.000000
Name: actions, dtype: float64

Среднее количество действий в тестовой группе примерно в 2 раза выше, чем в контрольной. Это значит, что пользователи из тестовой группы более активны, они ставят больше лайков и дизлайлов, чем польщователи из контрольной группы. Однако рано делать выводы, так как пока не доказана статистическая значимость подобных различий.

Судя по стандартному отклонению в обеих выборказ, метрики "среднее количество действий" в 0 и 1 группе имеют примерно одинаковые дисперсии. Проверим это с помощью критерия Левена:
* Н0 - дисперсии групп равны;
* Н1 - дисперсии групп не равны.

In [17]:
ss.levene(aggregate.query('group == 0').actions,
aggregate.query('group == 1').actions)

LeveneResult(statistic=0.4804538582845789, pvalue=0.48837808141783545)

pvalue > 0.05, значит у нас нет оснований отклонить нулевую гипотезу - дисперсии групп равны. 

Раз дисперсии равны, то используем t-критерия Стьюдента для проверки различий среднего количества действий в тестовой и контрольной группах на статистическую значимость:

* Н0 - различия в среднем количестве действий статистически не значимы;
* Н1 - различия в среднем количестве действий статистически значимы.

In [19]:
# Выполнение t-критерия Стьюдента
ss.ttest_ind(aggregate.query('group == 0').actions,
aggregate.query('group == 1').actions,
equal_var=True)

Ttest_indResult(statistic=-51.85383774946492, pvalue=1.8942877064043142e-285)

pvalue < 0.05, значит отклоняем нулевую гипотезу и делаем вывод, что среднее количество действий в группах статистически различается.

# Вывод
1. Средняя доля мэтчей в тестовой группе в два раза больше, чем в контрольной. 
    - Средняя доля мэтчей между двумя группами статистически различается.


2. Среднее количество действий в тестовой группе примерно в 2 раза выше, чем в контрольной. Это значит, что пользователи из тестовой группы более активны, они ставят больше лайков и дизлайлов, чем польщователи из контрольной группы. 
    - Среднее количество действий в группах статистически различается.


### Стоит ли включать новую систему поиска анкет на всех пользователей?

Да, потому что новая система увеличивает метрики. Результаты A/B-теста показали, что новая система улучшила долю мэтчей и активность клиентов в продукте.